In [ ]:
import pandas as pd 
import string
import nltk
import keras
import sklearn
import time
import matplotlib.pyplot as plt
import seaborn as snsimport numpy
import numpy as np
from time import time
from nltk.corpus import stopwords
import keras.preprocessing.text
import pandas as pd

In [ ]:
#pip install -U ipykernel


  Attempting uninstall: traitlets
    Found existing installation: traitlets 5.0.5
    Uninstalling traitlets-5.0.5:
      Successfully uninstalled traitlets-5.0.5
  Attempting uninstall: tornado
    Found existing installation: tornado 6.0.4
    Uninstalling tornado-6.0.4:
      Successfully uninstalled tornado-6.0.4



ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\ASUS\\anaconda3\\Lib\\site-packages\\~ornado\\speedups.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'C:\Users\ASUS\anaconda3\python.exe -m pip install --upgrade pip' command.


### Importing Aranews Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

df = pd.read_csv('/content/drive/MyDrive/Untitled folder/Arabic_pre3.csv',sep=',',error_bad_lines=False)
df.head()

,Unnamed: 0,Sentence,Label
0,329915,"مراكشتعددت , في ال+ آون +ة ال+ أخير +ة , جرائم...",0
1,76605,""" و+ بينما أضاف فرانسوا مول +ين , ال+ مدعي ال+...",0
2,92817,""" مصر لم تخسر نهائي "" كان "" منذ 88 عام +ا صدام...",0
3,120592,""" و+ أكد مسؤول رفيع في ال+ وكال +ة ال+ فضائي +...",1
4,71748,""" و+ قال +ت ال+ شرط +ة ال+ بريطاني +ة +ان عملي...",1


In [ ]:
df.Label.value_counts()

1    243635
0    243325
Name: Label, dtype: int64

In [ ]:
df = pd.read_csv(r"E:\My Data\Graduation project work\datasets\Arabic fake news dataset\ZeroShot_train_ArNews_5X.tsv",sep='\t',header=0)
df.head()

,Sentence,Label
0,"""وقال نقيب المحامين في كلمته بالمناسبة , اصالة...",Not_fake
1,نفذت سرقات في جنح الظلام استهدفت 888 راسا من ا...,Fake
2,ولم تتمكن الدراسة من تحديد مدة تعاطي المسكنات ...,Not_fake
3,وينتظر ان تكون الشرطة القضائية احالت المتهم عل...,Not_fake
4,واكد بنشماش علي الموقف الراسخ والثابت علي مر ا...,Not_fake


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 486960 entries, 0 to 486959
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Sentence  486297 non-null  object
 1   Label     486960 non-null  object
dtypes: object(2)
memory usage: 7.4+ MB


In [ ]:
df2=pd.read_csv('E:\My Data\Graduation project work\datasets\Arabic fake news dataset\Arabic_News.csv')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194784 entries, 0 to 194783
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Sentence  194645 non-null  object
 1   Label     194784 non-null  object
dtypes: object(2)
memory usage: 3.0+ MB


In [ ]:
df.columns

Index(['Sentence', 'Label'], dtype='object')

### Merging the datasets

In [ ]:
df_merge =pd.concat([df2, df], axis =0 )

In [ ]:
df_merge.head()

,Sentence,Label
0,وشارك في الندوة وزيرة السياحة والاثار رولا معا...,Not_fake
1,"وتحتل جهة الشاوية ورديغة , الرتبة الاولي في ال...",Fake
2,"""دول اختارت استراتيجية دقيقة لمواجهة الجوائح ر...",Fake
3,"وتابع : مع ذلك , نريد ان نتخذ هذه الخطوة .",Not_fake
4,"""ونبه المدير العام للمكتب الوطني المغربي للسيا...",Fake


In [ ]:
df_merge=df_merge.fillna(" ")

In [ ]:

df_merge.loc[(df_merge['Label'] == 'Not_fake'), ['Label']] = '1'
df_merge.loc[(df_merge['Label'] == 'Fake'), ['Label']] = '0'
df_merge.head()

,Sentence,Label
0,وشارك في الندوة وزيرة السياحة والاثار رولا معا...,1
1,"وتحتل جهة الشاوية ورديغة , الرتبة الاولي في ال...",0
2,"""دول اختارت استراتيجية دقيقة لمواجهة الجوائح ر...",0
3,"وتابع : مع ذلك , نريد ان نتخذ هذه الخطوة .",1
4,"""ونبه المدير العام للمكتب الوطني المغربي للسيا...",0


In [ ]:
df.drop_duplicates(inplace = True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 486960 entries, 0 to 486959
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Sentence  486297 non-null  object
 1   Label     486960 non-null  object
dtypes: object(2)
memory usage: 7.4+ MB


Fake    96426
Real    94112

In [ ]:
df_merge.Label.value_counts()

1    341089
0    340655
Name: Label, dtype: int64

In [ ]:
type(df_merge['Sentence'][2])

str

In [ ]:
df_merge=df_merge.reset_index(drop=True)

In [ ]:
df_merge['Sentence'][66]


'88 % , علي الرغم من انخفاض اسعار الفواكه الطازجة بمقدار 8 .'

In [ ]:
df['Label']

0         Not_fake
1             Fake
2         Not_fake
3         Not_fake
4         Not_fake
            ...   
486955        Fake
486956    Not_fake
486957        Fake
486958    Not_fake
486959    Not_fake
Name: Label, Length: 486960, dtype: object

In [ ]:
df['Sentence'][40]

'"واورد ابن خلدون في مقدمته اسباب لجوء عدد من المفسرين الي بعض اليهود واسفارهم لتفسير بعض القصص والاحداث , قائلا "" وقد جمع المتقدمون . . . , الا ان كتبهم ومنقولاتهم تشتمل علي الغث والسمين , والمقبول والمردود , والسبب في ذلك ان العرب لم يكونوا اهل كتاب ويحفظن علم , وانما غلبت عليهم البداوة والامية "" , ويضيف ابن خلدون "" واذا تشوقوا الي معرفة شيء مما تتشوق اليه النفوس البشرية في اسباب المكونات , وبدء الخليقة , واسرار الوجود , فانما يسالون عنها اهل الكتاب قبلهم , ويستفيدون "" ."'

## Text preprocessing in an Arabic text
#### includes functions for transliteration, stemming, normalization, and stop-word removal.

In [ ]:
from nltk.corpus import stopwords
from textblob import TextBlob
import re
from dsaraby import DSAraby
from tashaphyne.stemming import ArabicLightStemmer
from nltk.stem.isri import ISRIStemmer

# Load Arabic stop words
stops = set(stopwords.words("arabic"))
stop_word_comp = {"،", "آض", "آمينَ", "آه", "آهاً", "آي", "أ", "أب", "أجل", ...}

# Initialize Arabic stemming tools
ds = DSAraby()
ArListem = ArabicLightStemmer()

def to_arabic(text):
    """Transliterate text to Arabic."""
    return ds.transliterate(text)

def stem(text):
    """Apply Arabic stemming to a given text."""
    zen = TextBlob(text)
    words = zen.words
    cleaned = list()
    for w in words:
        ArListem.light_stem(w)
        cleaned.append(ArListem.get_root())
    return " ".join(cleaned)

def normalizeArabic(text):
    """Normalize Arabic text by removing diacritics and noise."""
    text = text.strip()
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(noise, '', text)
    text = re.sub(r'(.)\1+', r"\1\1", text) # Remove longation
    return araby.strip_tashkeel(text)

def remove_stop_words(text):
    """Remove Arabic stop words from a given text."""
    zen = TextBlob(text)
    words = zen.words
    return " ".join([w for w in words if not w in stops and not w in stop_word_comp and len(w) >= 2])


In [ ]:
def clean_tweet(text):
    text = re.sub('#\d+K\d+', ' ', text)  # years like 2K19
    text = re.sub('http\S+\s*', ' ', text)  # remove URLs
    text = re.sub('RT|cc', ' ', text)  # remove RT and cc
    text = re.sub('@[^\s]+',' ',text)
    text = clean_hashtag(text)
    #text = clean_emoji(text)
    return text

In [ ]:
from __future__ import unicode_literals
import re
import emoji
from aiogoogletrans import Translator
import asyncio

# Function to remove emojis from text
def remove_emoji(text):
    """
    Remove emojis from the given text.

    Parameters:
    - text (str): The input text.

    Returns:
    - str: Text with emojis removed.
    """
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    return text

# Function to translate emoji expressions to Arabic
def emoji_native_translation(text):
    """
    Translate native emoji expressions to Arabic equivalents.

    Parameters:
    - text (str): The input text.

    Returns:
    - str: Translated text.
    """
    text = text.lower()
    loves = ["<3", "♥", '❤']
    smilefaces = []
    sadfaces = []
    neutralfaces = []

    eyes = ["8", ":", "=", ";"]
    nose = ["'", "`", "-", r"\\"]
    # ... (rest of the code)

    return newText

# Function to check if a word is an emoji
def is_emoji(word):
    """
    Check if a word is an emoji.

    Parameters:
    - word (str): The word to check.

    Returns:
    - bool: True if the word is an emoji, False otherwise.
    """
    if word in emojis_ar:
        return True
    else:
        return False

# Function to add space around emojis
def add_space(text):
    """
    Add space around emojis in the given text.

    Parameters:
    - text (str): The input text.

    Returns:
    - str: Text with spaces around emojis.
    """
    return ''.join(' ' + char if is_emoji(char) else char for char in text).strip()

# Function to clean emojis in a text
def clean_emoji(text):
    """
    Clean emojis in the given text.

    Parameters:
    - text (str): The input text.

    Returns:
    - str: Cleaned text.
    """
    text = emoji_native_translation(text)
    text = remove_emoji(text)
    return text

# Initialize Translator
translator = Translator()
loop = asyncio.get_event_loop()

In [ ]:
def clean_text(text):
    ## Clean for tweets
    text = clean_tweet(text)
    ## Remove punctuations
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)  # remove punctuation
    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)  
    ## Remove Emojis
    #text = remove_emoji(text)
    ## Convert text to lowercases
    text = text.lower()
    ## Arabisy the text
    text = to_arabic(text)
    ## Remove stop words
    text = remove_stop_words(text)
    ## Remove numbers
    text = re.sub("\d+", " ", text)
    ## Remove Tashkeel
    text = normalizeArabic(text)
    #text = re.sub('\W+', ' ', text)
    text = re.sub('[A-Za-z]+',' ',text)
    text = re.sub(r'\\u[A-Za-z0-9\\]+',' ',text)
    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)  
    #Stemming
    text = stem(text)
    return text

In [ ]:
#Applying the preprocessing functions
df_merge['Sentence'] = df_merge['Sentence'].apply(lambda x:clean_text(x))

### Sample of preprocessed text

In [ ]:
for i in range(10):
    print(df_merge['Sentence'][i])
    print(df_merge["Label"][i])

وشارك الندوه وزيره السياحه والاثار رولا معايعه ووزير شءون القدس عدنان الحسيني ومفتي القدس والديار الفلسطيني الشيخ محمد حسين وقاضي القضاه الشرعي الدكتور محمود الهباش والدكتور تيسير جردات وكيل وزاره الخارجيه والمطران عطا الله حنا وسفيري الاردن والمغرب اضافه الي ممثل منظمه التعاون الاسلامي لدي دوله فلسطين السفير احمد الرويضي وعدد ممثلي المءسسات السياحيه الفلسطينيه القدس
1
وتحتل جهه الشاويه ورديغه الرتبه الاولي الحالات المستقبله قق حاله بينها قق امراه تعرضن لعنف اقتصادي قق حالات اغتصاب متبوعه بجهه تادله ازيلال ققخ حاله بينها حالات اغتصاب متعلقه بالتحرش الجنسي قق حاله حرمان النفقه وجهه تازه الحسيمه تاونات قق حالات قق تعرضن الي عنف جسدي قق حاله لعنف يستقبل المركز جهه العيون بوجدور الساقيه الحمراء سوي قق نساء معنفات بينها حالتا اغتصاب وتحرش جنسي قق حاله عنف اقتصادي قق امراه حرمت النفقه وعدد مماثل تعرضن الي عنف قانوني قق حاله عنف جسدي سجلت بجهات وادي الذهب وكلميم السماره وسوس ماسه درعه تباعا قق قق قق حاله عنف
0
دول اختارت استراتيجيه دقيقه لمواجهه الجواءح رغم المغرب يسجل تقدما علي مستوي الاجهزه

### Save cleaned data in CSV format

In [ ]:
df.to_csv(r'E:\My Data\Graduation project work\datasets\Arabic fake news dataset\Arabic_pre2.csv')

In [ ]:
dff= pd.read_csv('/content/drive/MyDrive/Untitled folder/Arabic_pre2.csv')

In [ ]:
dff=dff.reset_index(drop=True)

In [ ]:
dff = dff.sample(frac = 1)
dff.loc[(dff['Label'] == 'Not_fake'), ['Label']] = '1'
dff.loc[(dff['Label'] == 'Fake'), ['Label']] = '0'
dff.head()

,Unnamed: 0,Sentence,Label
439237,439237,"""لكن المؤسسة الوطنية للنفط التي في الشرق قالت ...",0
433696,433696,قرار المجلس الدستوري وضمانات المحاكمة العادلة ...,0
62002,62002,"8 مليون اورو عن كل فوز , و 895 الف اورو عن كل ...",0
16600,16600,وامر الملك سلمان بن عبد العزيز عاهل السعودية ا...,1
75465,75465,"""وقال """" انضم اليوم في افغانستان الي الصائمين ...",1
